In [2]:
# Imports
import numpy as np
from sympy import symbols, solve, Poly, simplify, latex, Rational, init_printing
import random
import cmath
import pandas as pd
from tqdm import tqdm

init_printing(use_latex=True)
from io import StringIO

---
# Problem type 1: Nondimensionalization of a polynomial with random powers (symbolic answer)

For our first type of problem, we choose to do nondimensionalization problems of the format $a_1 x^{n_1} + a_2 x^{n_2} + a_3$ where $n_1 > n_2$


The expected form of the final answer will be $\epsilon y^{n_1} + y^{n_2} + 1$

In [3]:
# Generates two numbers n1, n2 < max_degree where n1 < n2
def generate_n1n2(max_degree):
    """
    Inputs:
    max_degree (int): Maximum possible degree of polynomial

    Outputs:
    n1 (int): Degree of polynomial
    n2 (int): Second largest power of polynomial
    """
    n2 = random.randint(1, max_degree - 1)
    n1 = random.randint(n2 + 1, max_degree)
    return n1, n2

In [110]:
# Solves nondimensionalization problem given n1, n2
def nondimensionalize_polynomial1(n1, n2):
    """
    Inputs:
    n1 (int): Degree of polynomial
    n2 (int): Second largest power of polynomial

    Outputs:
    question (string): Latex for problem
    answer (string): Latex for answer
    """
    # Define sympy symbols
    x, y, a1, a2, a3, epsilon = symbols('x y a_1 a_2 a_3 epsilon')

    # Generate sympy polynomial a1*x^n1 + a2*x^n2 + a3
    n1r, n2r = Rational(n1), Rational(n2)
    polynomial_x = a1 * x**n1r + a2 * x**n2r + a3

    # Define x = (a/b)y
    x_sub = ((a3/a2)**Rational(1/n2r)) * y

    # Substitute
    polynomial_y = polynomial_x.subs(x, x_sub).expand()
    # polynomial_y = polynomial_y.subs(x, y).expand()

    # Simplify before dividing by a3
    simplified_polynomial = polynomial_y / a3

    # Further simplify the expression
    final_simplified_polynomial = simplify(simplified_polynomial)

    # Return question and answer
    question = ""
    answer = ""

    question += "Nondimensionalize the following polynomial"
    question += "\[" + latex(polynomial_x) + "\]"
    question += f"into one of the form $\epsilon y^{{{n1}}} + y^{{{n2}}} + 1$"

    answer += "Start with the following substitution:"
    answer += "\[" + "x=" + latex(x_sub) + "\]"

    answer += "\nThis gives us the following expression:"
    answer += "\[" + latex(polynomial_y) + "\]"

    answer += "\nDivide by the coefficient remaining in front of the constant, giving us the nondimensionalized polynomial with coefficients in terms of $a_1, a_2, a_3$:"
    answer += "\[ \\boxed{" + latex(final_simplified_polynomial) + "}\]"

    return question, answer

In [14]:
# n1, n2 = generate_n1n2(max_degree=10)
# question, answer = nondimensionalize_polynomial1(n1, n2)

question, answer = nondimensionalize_polynomial1(n1=10, n2=4) # Problem from hw1 to check

print(question)
print(answer)


Nondimensionalize the following polynomial\[a_{1} x^{10} + a_{2} x^{4} + a_{3}\]into one of the form $\epsilon y^{10} + y^{4} + 1$

Start with the following substitution:\[x=y \sqrt[4]{\frac{a_{3}}{a_{2}}}\]
This gives us the following expression:\[a_{1} y^{10} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{5}{2}} + a_{3} y^{4} + a_{3}\]
Divide by the coefficient remaining in front of the constant, giving us the nondimensionalized polynomial with coefficients in terms of $a_1, a_2, a_3$:\[ \boxed{\frac{a_{1} y^{10} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{5}{2}}}{a_{3}} + y^{4} + 1}\]


---
# Problem type 2: Nondimensionalization of a polynomial with random powers AND coefficients

The second type of problem is the same as the first, but with a random polynomial (coefficients provided instead of a1, a2, and a3, can also be +/-). The goal is then to nondimensionalize it in the same way, and solve for the value of epsilon.

In [17]:
# Generates a list of coefficients corresponding to a random polynomial
def generate_polynomial(max_degree, num_terms, coeff_bounds):
    """
    Inputs:
    max_degree (int): The maximum degree of the polynomial.
    num_terms (int)
    coeff_bounds (tuple): (lower bound, upper bound)

    Outputs:
    coefficients (list)
    """
    # Initialize array of coeffs
    coefficients = [0] * (max_degree + 1)
    # Check number of terms
    if num_terms > max_degree + 1:
        raise ValueError("Number of terms cannot be more than the degree of the polynomial + 1")
    # Randomly choose positions for the (#) non-zero coefficients
    non_zero_positions = random.sample(range(max_degree), num_terms - 1)
    # Assign random values within the bounds to the chosen positions
    for pos in non_zero_positions:
        # Loop to make sure it is nonzero bc selecting from [-, +] includes 0
        while coefficients[pos] is None or coefficients[pos] == 0:
          coefficients[pos] = random.randint(coeff_bounds[0], coeff_bounds[1])
    # Ensure the highest degree term is non-zero
    if coefficients[-1] == 0:
        # While loop until it is nonzero
        while coefficients[-1] is None or coefficients[-1] == 0:
          coefficients[-1] = random.randint(coeff_bounds[0], coeff_bounds[1])

    return coefficients

In [69]:
# Test function
coefficients = generate_polynomial(max_degree=10, num_terms=3, coeff_bounds=[-10,10])
coefficients

[0, 1, 0, 0, 0, 0, -10, 0, 0, 0, -8]

In [37]:
# Helper function to convert list of coefficients into a sympy expression
def sympy_polynomial_from_coefficients(coefficients):
    """
    Inputs:
    coefficients (list): A list of coefficients, where the index represents the power of x.

    Outputs:
    polynomial (sympy expression): The polynomial expression.
    """
    x = symbols('x')
    coefficients_reverse = coefficients
    coefficients_reverse.reverse()
    polynomial = sum(coef * x**i for i, coef in enumerate(coefficients_reverse))
    return polynomial

In [39]:
# Test helper function
sympy_polynomial_from_coefficients(coefficients)

     10      4      2
- 6⋅x   - 6⋅x  - 3⋅x 

In [111]:
# Solves nondimensionalization problem given coefficients
def nondimensionalize_polynomial2(coefficients):
    """
    Inputs:
    coefficients (list): The list of coefficients

    Outputs:
    question (string): Latex for problem
    answer (string): Latex for answer
    """
    question = ""
    answer = ""

    # Find nonzero coefficients & powers in the format [(coefficient, power)]
    nonzero_coeffs = [(coeff, len(coefficients) - idx - 1) for idx, coeff in enumerate(coefficients) if coeff != 0]
    # Find largest, second largest powers
    sorted_coeffs = sorted(nonzero_coeffs, key=lambda x: x[1], reverse=True)
    n1 = sorted_coeffs[0][1]
    n2 = sorted_coeffs[1][1]
    sign1 = np.sign(nonzero_coeffs[0][0])
    sign2 = np.sign(nonzero_coeffs[1][0])
    sign3 = np.sign(nonzero_coeffs[2][0])
    a1_absvalue = abs(nonzero_coeffs[0][0])
    a2_absvalue = abs(nonzero_coeffs[1][0])
    a3_absvalue = abs(nonzero_coeffs[2][0])

    # Debugging delete later
    print(n1, n2)
    print(sign1, sign2, sign3)
    print(a1_absvalue, a2_absvalue, a3_absvalue)

    # Define sympy symbols
    x, y, a1, a2, a3, epsilon = symbols('x y a_1 a_2 a_3 epsilon')

    # Generate sympy polynomial a1*x^n1 + a2*x^n2 + a3
    n1r, n2r = Rational(n1), Rational(n2)
    polynomial_x = sign1*a1 * x**n1r + sign2*a2 * x**n2r + sign3*a3 # included signs

    answer += "\nSet aside the coefficient values and use $a_1, a_2, a_3$ for now. Our polynomial is then:"
    answer += "\[" + latex(polynomial_x) + "\]"

    # Branching for if first coefficient is negative
    if sign1 < 0:
        polynomial_x *= (-1)
        answer += "\nSince the first coefficient is negative, we multiply the entire expression by -1:"
        answer += "\[" + latex(polynomial_x) + "\]"

    # Define x = (a/b)y
    x_sub = ((a3/a2)**Rational(1/n2r)) * y

    # Substitute
    polynomial_y = polynomial_x.subs(x, x_sub).expand()

    # Simplify before dividing by a3
    simplified_polynomial = polynomial_y / a3

    # Further simplify the expression
    final_simplified_polynomial = simplify(simplified_polynomial)

    # Substitute know values for a1, a2, a3
    final_simplified_polynomial_subs = final_simplified_polynomial.subs([(a1, a1_absvalue), (a2, a2_absvalue), (a3, a3_absvalue)]).simplify()

    # Find value of epsilon
    epsilon_value = list(final_simplified_polynomial_subs.expand().args)[::-1][0] / (y**n1)

    # Return question and answer
    question += "Nondimensionalize the following polynomial"
    question += "\[" + latex(sympy_polynomial_from_coefficients(coefficients)) + "\]"
    question += f"into one of the form $\epsilon y^{{{n1}}} \pm y^{{{n2}}} \pm 1$. Solve for epsilon."

    answer += "Use the following substitution:"
    answer += "\[" + "x=" + latex(x_sub) + "\]"

    answer += "\nThis gives us the following expression:"
    answer += "\[" + latex(polynomial_y) + "\]"

    answer += "\nDivide by the coefficient remaining in front of the constant, giving us the nondimensionalized polynomial with coefficients in terms of $a_1, a_2, a_3$:"
    answer += "\[" + latex(final_simplified_polynomial) + "\]"

    answer += "\nSubstituting the known values for $a_1, a_2, a_3$ (absolute values as we already accounted for sign):"
    answer += "\[" + latex(final_simplified_polynomial_subs) + "\]"

    answer += "\nFrom inspection of our nondimensionalized equation, we can identify $\epsilon$:"
    answer += "\[ \\boxed{\epsilon=" + latex(epsilon_value) + "}" + f"\\approx{epsilon_value.evalf():.2f} \]"

    return question, answer

In [95]:
print(coefficients)
question, answer = nondimensionalize_polynomial2(coefficients)

print(question)
print(answer)

[-8, 0, 0, 0, -10, 0, 0, 0, 0, 1, 0]
10 6
-1 -1 1
8 10 1

Nondimensionalize the following polynomial\[- 8 x^{10} - 10 x^{6} + x\]into one of the form $\epsilon y^{10} \pm y^{6} \pm 1$. Solve for epsilon.

Set aside the coefficient values and use $a_1, a_2, a_3$ for now. Our polynomial is then:\[- a_{1} x^{10} - a_{2} x^{6} + a_{3}\]
Since the first coefficient is negative, we multiply the entire expression by -1:\[a_{1} x^{10} + a_{2} x^{6} - a_{3}\]
Use the following substitution:\[x=y \sqrt[6]{\frac{a_{3}}{a_{2}}}\]
This gives us the following expression:\[a_{1} y^{10} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{5}{3}} + a_{3} y^{6} - a_{3}\]
Divide by the coefficient remaining in front of the constant, giving us the nondimensionalized polynomial with coefficients in terms of $a_1, a_2, a_3$:\[\frac{a_{1} y^{10} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{5}{3}}}{a_{3}} + y^{6} - 1\]
Substituting the known values for $a_1, a_2, a_3$ (absolute values as we already accounted for sign):\[\frac{

---
# Problem type 3: Solving for roots

Given a nondimensionalized problem of form
$\epsilon x^{n_1} \pm x^{n_2} \pm 1$ where $n_1 > n_2$, find the roots in terms of epsilon (using dominant balances) for both large and small $\epsilon$.

In [102]:
# Generates nondimensionalized polynomial of the form described above
def generate_nondimensionalized(max_n1):
    """
    Inputs:
    max_n1 (int): Maximum power in polynomial

    Outputs:
    polynomial (sympy expression): Nondimensionalized random polynomial of the given format
    """
    # Ensure max_n1 is at least 2
    if max_n1 < 2:
        raise ValueError("max_n1 must be at least 2")

    # Randomly choose n1 and n2
    n1 = random.randint(2, max_n1)
    n2 = random.randint(1, n1 - 1)
    # Randomly choose signs
    signs = random.choices([-1, 1], k=2)

    # Construct polynomial, let first term always be + since same as negating if negative
    x = symbols('x')
    epsilon = symbols('epsilon')
    polynomial = epsilon * x**n1 + signs[0] * x**n2 + signs[1]

    # Return
    return polynomial

In [137]:
# Example usage of random polynomial generator
polynomial = generate_nondimensionalized(10)
display(polynomial)

   5        
ε⋅x  - x - 1

In [163]:
# Function to solve for roots of nondimensionalized polynomial
def solve_roots(polynomial):
    """
    Inputs:
    polynomial (sympy expression): The nondimensionalized polynomial

    Outputs:
    question (string): Latex for problem
    answer (string): Latex for answer
    """
    # Extract terms and solve
    x, epsilon = symbols('x epsilon')
    terms = list(polynomial.expand().args)[::-1]
    A, B, C = terms[0], terms[1], terms[2]
    sol_ab = [simplify(sol) for sol in solve(A + B, x)]
    sol_bc = [simplify(sol) for sol in solve(B + C, x)]
    sol_ac = [simplify(sol) for sol in solve(A + C, x)]

    # Remove extraneous root 0 that shows up - bc it never occurs in this problem formulation
    # Helper function to remove zeros
    def remove_zeros(sol_list):
        while 0 in sol_list:
            sol_list.remove(0)
    remove_zeros(sol_ab)
    remove_zeros(sol_bc)
    remove_zeros(sol_ac)

    # Check dominant balances to see if roots belong to small or large epsilon regimes
    AB_valid_small_eps = (abs(A.subs(x, sol_ab[0]).subs(epsilon, 0.0001)) > abs(C.subs(x, sol_ab[0]).subs(epsilon, 0.001))) # A,B >> C small eps
    AB_valid_large_eps = (abs(A.subs(x, sol_ab[0]).subs(epsilon, 10000)) > abs(C.subs(x, sol_ab[0]).subs(epsilon, 1000))) # A,B >> C large eps
    AB_validity = "small" if AB_valid_small_eps else "large"

    BC_valid_small_eps = (abs(B.subs(x, sol_bc[0]).subs(epsilon, 0.0001)) > abs(A.subs(x, sol_bc[0]).subs(epsilon, 0.001))) # B,C >> A small eps
    BC_valid_large_eps = (abs(B.subs(x, sol_bc[0]).subs(epsilon, 10000)) > abs(A.subs(x, sol_bc[0]).subs(epsilon, 1000))) # B,C >> A large eps
    BC_validity = "small" if BC_valid_small_eps else "large"

    AC_valid_small_eps = (abs(A.subs(x, sol_ac[0]).subs(epsilon, 0.0001)) > abs(B.subs(x, sol_ac[0]).subs(epsilon, 0.001))) # A,C >> B small eps
    AC_valid_large_eps = (abs(A.subs(x, sol_ac[0]).subs(epsilon, 10000)) > abs(B.subs(x, sol_ac[0]).subs(epsilon, 1000))) # A,C >> B large eps
    AC_validity = "small" if AC_valid_small_eps else "large"


    # Return answers
    question = ""
    answer = ""

    question += "Solve for the roots of the following polynomial in terms of $\epsilon$:"
    question += "\[" + latex(polynomial) + "\]"
    question += "Use the method of dominant balances and find approximations of roots for both small and large $\epsilon$"

    answer += "Consider the problem to be $A+B+C=0$, with A,B, and C corresponding to our three terms."
    answer += "\[ A=" + latex(A) + "\]"
    answer += "\[ B=" + latex(B) + "\]"
    answer += "\[ C=" + latex(C) + "\]"

    answer += "\nWe will consider the three possible dominant balances, solving for the roots of each and evaluating if the balances hold for large or small $\epsilon$."
    answer += "\\vspace{1em}"

    answer += "\n\nWe start with the balance $A+B=0$, solving for x in terms of $\epsilon$ gives us"
    answer += f"\n{len(sol_ab)} roots:"
    answer += "\[" + latex(A+B) + "=0\]"
    answer += "\[ \implies\\boxed{ x=" + latex(sol_ab) + "}\]"
    answer += "\nChecking the consistency that $|A|,|B|>>|C|$, we substitute the root back into A, B, and C to check their magnitudes:"
    answer += f"\n\nValidity for small $\epsilon$: {str(AB_valid_small_eps)}"
    answer += f"\n\nValidity for large $\epsilon$: {str(AB_valid_large_eps)}"
    answer += f"\n\n\\underline{{Therefore these are roots for {AB_validity} $\epsilon$}}"
    answer += "\\vspace{1em}"

    answer += "\n\nNext we use the balance $B+C=0$, solving for x in terms of $\epsilon$ gives us"
    answer += f"\n{len(sol_bc)} roots:"
    answer += "\[" + latex(B+C) + "=0\]"
    answer += "\[ \implies\\boxed{ x=" + latex(sol_bc) + "}\]"
    answer += "\nChecking the consistency that $|B|,|C|>>|A|$, we substitute the root back into A, B, and C to check their magnitudes:"
    answer += f"\n\nValidity for small $\epsilon$: {str(BC_valid_small_eps)}"
    answer += f"\n\nValidity for large $\epsilon$: {str(BC_valid_large_eps)}"
    answer += f"\n\n\\underline{{Therefore these are roots for {BC_validity} $\epsilon$}}"
    answer += "\\vspace{1em}"

    answer += "\n\nNext we use the balance $A+C=0$, solving for x in terms of $\epsilon$ gives us"
    answer += f"\n{len(sol_ac)} roots:"
    answer += "\[" + latex(A+C) + "=0\]"
    answer += "\[ \implies\\boxed{ x=" + latex(sol_ac) + "}\]"
    answer += "\nChecking the consistency that $|A|,|C|>>|B|$, we substitute the root back into A, B, and C to check their magnitudes:"
    answer += f"\n\nValidity for small $\epsilon$: {str(AC_valid_small_eps)}"
    answer += f"\n\nValidity for large $\epsilon$: {str(AC_valid_large_eps)}"
    answer += f"\n\n\\underline{{Therefore these are roots for {AC_validity} $\epsilon$}}"
    answer += "\\vspace{1em}"

    answer += "\n\nCounting the total number of roots we have found, we count a total of"
    answer += f"\n{len(sol_ab) + len(sol_bc) + len(sol_ac)} roots which we expect given the degree of our polynomial."

    return question, answer

In [164]:
# Test root solver function
polynomial = generate_nondimensionalized(4)
question, answer = solve_roots(polynomial)

print(question)
print(answer)

Solve for the roots of the following polynomial in terms of $\epsilon$:\[\epsilon x^{2} - x + 1\]Use the method of dominant balances and find approximations of roots for both small and large $\epsilon$
Consider the problem to be $A+B+C=0$, with A,B, and C corresponding to our three terms.\[ A=\epsilon x^{2}\]\[ B=- x\]\[ C=1\]
We will consider the three possible dominant balances, solving for the roots of each and evaluating if the balances hold for large or small $\epsilon$.\vspace{1em}

We start with the balance $A+B=0$, solving for x in terms of $\epsilon$ gives us
1 roots:\[\epsilon x^{2} - x=0\]\[ \implies\boxed{ x=\left[ \frac{1}{\epsilon}\right]}\]
Checking the consistency that $|A|,|B|>>|C|$, we substitute the root back into A, B, and C to check their magnitudes:

Validity for small $\epsilon$: True

Validity for large $\epsilon$: False

\underline{Therefore these are roots for small $\epsilon$}\vspace{1em}

Next we use the balance $B+C=0$, solving for x in terms of $\epsilon$ 

---
# Generate problems, save results to pandas dataframe, export to excel in desired format

In [200]:
# Number of desired problems
n_problems = 10

# Create empty dataframe
df = pd.DataFrame(columns=['Question', 'Answer'])

# Iterate once for each problem
for i in tqdm(range(n_problems)):
    # Generate new problem
    coefficients = generate_polynomial(max_degree=10, num_terms=3, coeff_bounds=[-10,10])
    # Solve new problem for new question, answer
    new_question, new_answer = nondimensionalize_polynomial2(coefficients)
    # Add new question answer pair to data
    row_df = pd.DataFrame({'Question': [new_question], 'Answer': [new_answer]})
    df = pd.concat([df, row_df], ignore_index=True)

# Save dataframe to excel file of specified format
df.to_excel('polynomial_dataset.xlsx', index=False)

### Old (delete?)

In [ ]:
# Define the file name
file_name = "problem_dataset.tex"

# Write to file
with open(file_name, "w") as file:
    file.write("\\documentclass{article}\n")
    file.write("\\usepackage{amsmath}\n")
    file.write("\\begin{document}\n")
    file.write("\\section*{Quadratic Equation}\n")
    file.write("The quadratic equation is given by:\n")
    file.write("\\[" + latex(polynomial) + "\\]\n")
    file.write("\\end{document}")